In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F


# Initialize SparkSession
spark = SparkSession.builder.appName("Test").getOrCreate()

# Test Spark context
# print(spark.sparkContext.getConf().getAll())

In [1]:
# Load the updated data from HDFS
#df = spark.read.csv("hdfs:///user/thokozile/outputMetaData", header=False, inferSchema=True)
df = spark.read.csv("hdfs:///user/thokozile/outputMetaData/part-r*", header=False, inferSchema=True)

# Show the data
df.show()


In [2]:
columns = ["_c0", "_c1", "_c2", "_c3", "_c4","_c5"]

# Clean and split the DataFrame using regex
final_df = df.select(
    F.regexp_extract(F.trim(df["_c0"]), r"([\w\s]+)\s+([0-9.]+)", 1).alias("Product"),  # get product name
    F.regexp_extract(F.trim(df["_c0"]), r"([\w\s]+)\s+([0-9.]+)", 2).cast("double").alias("Price"),  # get price and convert to double
    *[F.trim(df[col]).alias(col) for col in columns[1:]]
)

In [3]:
# Show the final cleaned DataFrame
# Count the number of entries in the DataFrame
row_count = final_df.count()

print(f"Number of entries in the DataFrame: {row_count}")
final_df.show(truncate=False)

In [4]:
from pyspark.sql.functions import col
final_df = final_df.withColumnRenamed("_c1", "TopColour1") \
       .withColumnRenamed("_c2", "TopColour2") \
       .withColumnRenamed("_c3", "DominantSeason") \
       .withColumnRenamed("_c4", "DominantGender") \
       .withColumnRenamed("_c5", "ItemsCount")

final_df = final_df.withColumn('ItemsCount', col('ItemsCount').cast('int'))
final_df = final_df.withColumn('Price', col('Price').cast('float'))

final_df.show()

In [5]:
pandas_df = final_df.toPandas()

In [6]:
# Select top 10 products by average price
top_n = pandas_df.groupby("Product")["Price"].mean().nlargest(15).index
top_n_df = pandas_df[pandas_df["Product"].isin(top_n)]
custom_palette = sns.color_palette(["#2ecc71", "#3498db", "#e74c3c", "#9b59b6", "#f1c40f", "#e67e22", "#1abc9c", "#ff6347", "#8e44ad", "#bdc3c7","#34495e", "#d35400", "#16a085", "#c0392b", "#f39c12"])

# Plot only the top 15 products
plt.figure(figsize=(16, 8))
sns.barplot(x="Product", y="Price", data=top_n_df, hue="Product", palette=custom_palette, legend=False)
plt.xticks(rotation=45)
plt.title("Average Price for Top 15 Article Types")
plt.savefig('/mnt/Results/Average_Price.png', bbox_inches='tight')

plt.show()

In [ ]:
# Select bottom 10 products by average price
top_n = pandas_df.groupby("Product")["Price"].mean().nsmallest(15).index
top_n_df = pandas_df[pandas_df["Product"].isin(top_n)]
custom_palette = sns.color_palette(["#2ecc71", "#3498db", "#e74c3c", "#9b59b6", "#f1c40f", "#e67e22", "#1abc9c", "#ff6347", "#8e44ad", "#bdc3c7","#34495e", "#d35400", "#16a085", "#c0392b", "#f39c12"])

# Plot only the top 15 products
plt.figure(figsize=(16, 8))
sns.barplot(x="Product", y="Price", data=top_n_df, hue="Product", palette=custom_palette, legend=False)
plt.xticks(rotation=45)
plt.title("Average Price for Top 15 Most Frequent Products")
plt.savefig('/mnt/Results/Average_Price.png', bbox_inches='tight')

plt.show()

In [7]:
# Grouping by DominantGender to sum the ItemsCount
gender_counts = final_df.groupby('DominantGender').sum('ItemsCount')
gender_counts = gender_counts.toPandas()

print(gender_counts)


In [8]:
# Pie chart
colours = ['#3498db','#e74c3c','#ff69b4', '#9b59b6']

plt.figure(figsize=(8, 8))
plt.pie(gender_counts['sum(ItemsCount)'], labels=gender_counts['DominantGender'], autopct='%1.1f%%', startangle=90, colors=colours)
plt.title('Item Count Distribution by Gender')
plt.savefig('/mnt/Results/Gender_distribution.png', bbox_inches='tight')

plt.show()


In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

combined_colours = pd.concat([pandas_df['TopColour1'], pandas_df['TopColour2']])
colour_counts = combined_colours.value_counts().reset_index()
colour_counts.columns = ['Colour', 'Count']

top_colours = colour_counts.nlargest(15, 'Count')['Colour']

# Filter the original DataFrame for only the top 10 colors
filtered_df = pandas_df[(pandas_df['TopColour1'].isin(top_colours)) | (pandas_df['TopColour2'].isin(top_colours))]

# Melt the filtered DataFrame to long format
melted_df = filtered_df.melt(id_vars=['DominantSeason'], value_vars=['TopColour1', 'TopColour2'], var_name='ColourType', value_name='Colour')
melted_df = melted_df[melted_df['Colour'].isin(top_colours)]

# Set the figure size
plt.figure(figsize=(15, 8))

# Create a count plot with hue for DominantSeason
sns.countplot(data=melted_df, x='Colour', hue='DominantSeason', palette='viridis')

# Set plot labels and title
plt.title('Count of Items per Top Colour by Season (Top 15 Colours)')
plt.xlabel('Top Colour')
plt.ylabel('Count of Items')
plt.xticks(rotation=45)
plt.legend(title='Season')
plt.savefig('/mnt/Results/Season_colours.png', bbox_inches='tight')

# Display the plot
plt.show()
